In [52]:
#モデルの定義まではLightGBMの場合と同様
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [53]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [54]:
train_hp_dummies = pd.get_dummies(train_data['HomePlanet'], prefix='HomePlanet')
train_data = pd.concat([train_data, train_hp_dummies], axis=1)
 
test_hp_dummies = pd.get_dummies(test_data['HomePlanet'], prefix='HomePlanet')
test_data = pd.concat([test_data, test_hp_dummies], axis=1)

In [55]:
train_data[['Deck', 'Num', 'Side']] = train_data['Cabin'].str.split('/', expand=True) 

dummies = pd.get_dummies(train_data['Deck'], prefix='Deck_Type')
train_data = pd.concat([train_data,dummies], axis=1)

train_data['Side'] = train_data['Side'].map({'P': 0, 'S': 1})


test_data[['Deck', 'Num', 'Side']] = test_data['Cabin'].str.split('/', expand=True) 

dummies = pd.get_dummies(test_data['Deck'], prefix='Deck_Type')
test_data = pd.concat([test_data,dummies], axis=1)

test_data['Side'] = test_data['Side'].map({'P': 0, 'S': 1})

In [56]:
train_data['CryoSleep'] = train_data['CryoSleep'].astype(bool)
train_data['VIP'] = train_data['VIP'].astype(bool)

test_data['CryoSleep'] = test_data['CryoSleep'].astype(bool)
test_data['VIP'] = test_data['VIP'].astype(bool)

In [57]:
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Num,Side,Deck_Type_A,Deck_Type_B,Deck_Type_C,Deck_Type_D,Deck_Type_E,Deck_Type_F,Deck_Type_G,Deck_Type_T
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,...,0,0.0,False,True,False,False,False,False,False,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,...,0,1.0,False,False,False,False,False,True,False,False
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,...,0,1.0,True,False,False,False,False,False,False,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,...,0,1.0,True,False,False,False,False,False,False,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,...,1,1.0,False,False,False,False,False,True,False,False


In [58]:
test_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Num,Side,Deck_Type_A,Deck_Type_B,Deck_Type_C,Deck_Type_D,Deck_Type_E,Deck_Type_F,Deck_Type_G,Deck_Type_T
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,...,3,1.0,False,False,False,False,False,False,True,False
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,...,4,1.0,False,False,False,False,False,True,False,False
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,...,0,1.0,False,False,True,False,False,False,False,False
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,...,1,1.0,False,False,True,False,False,False,False,False
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,...,5,1.0,False,False,False,False,False,True,False,False


In [59]:
y_train_val = train_data['Transported']
x_train_val= train_data.drop(['PassengerId', 'HomePlanet', 'Cabin', 'Destination', 'Name', 'Transported','Deck', 'Deck_Type_T'], axis=1)

x_test= test_data.drop(['PassengerId', 'HomePlanet', 'Cabin', 'Destination', 'Name','Deck', 'Deck_Type_T'], axis=1)

In [60]:
x_train_val['Num'] = x_train_val['Num'].fillna(0).astype(int)
x_train_val['Num_Group'] = x_train_val['Num'] // 100
x_train_val = x_train_val.drop(['Num'], axis =1)

x_test['Num'] = x_test['Num'].fillna(0).astype(int)
x_test['Num_Group'] = x_test['Num'] // 100
x_test = x_test.drop(['Num'], axis =1)

In [61]:
from sklearn.model_selection import train_test_split

In [62]:
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.3, random_state=1)

In [63]:
xgb_model = xgb.XGBClassifier(n_estimators=500, learning_rate=0.05, use_label_encoder=False, eval_metric='logloss', random_state=42)
#モデルを任意のパラメータとともに定義

In [64]:
xgb_model.fit(x_train, y_train)
#モデルを適用

/Users/sei/Downloads/Kaggle/Space titanic/.venv/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [14:41:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [65]:
y_pred_xgb = xgb_model.predict(x_val)
print("Validation Accuracy:", accuracy_score(y_val, y_pred_xgb))
#validationデータでのスコアを確認

Validation Accuracy: 0.803680981595092


In [66]:
import optuna
from xgboost import XGBClassifier

In [67]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
    }
    model = XGBClassifier(**params, random_state=42)
    model.fit(x_train, y_train) 
    y_pred = model.predict(x_val)
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy
#Optunaで探索範囲を定義


In [68]:
study = optuna.create_study(direction="maximize")  # 精度を最大化
study.optimize(objective, n_trials=50)  # 50回試行

[I 2025-03-02 14:41:59,386] A new study created in memory with name: no-name-6ec04b8a-09ab-453e-a5f2-245d06d1fbaa
[I 2025-03-02 14:42:00,159] Trial 0 finished with value: 0.7956288343558282 and parameters: {'n_estimators': 235, 'max_depth': 14, 'learning_rate': 0.17267708787266378, 'subsample': 0.9846236125074284, 'colsample_bytree': 0.9861014014839664}. Best is trial 0 with value: 0.7956288343558282.
[I 2025-03-02 14:42:00,568] Trial 1 finished with value: 0.8009969325153374 and parameters: {'n_estimators': 336, 'max_depth': 5, 'learning_rate': 0.25271333510442606, 'subsample': 0.6516657076962281, 'colsample_bytree': 0.5148158722709979}. Best is trial 1 with value: 0.8009969325153374.
[I 2025-03-02 14:42:01,533] Trial 2 finished with value: 0.8006134969325154 and parameters: {'n_estimators': 273, 'max_depth': 15, 'learning_rate': 0.16771015703788242, 'subsample': 0.8007632147975119, 'colsample_bytree': 0.8924560927050317}. Best is trial 1 with value: 0.8009969325153374.
[I 2025-03-02 

In [69]:
print("Best hyperparameters:", study.best_params)
#スコアが一番良いパラメータを確認

Best hyperparameters: {'n_estimators': 337, 'max_depth': 8, 'learning_rate': 0.010880545726085105, 'subsample': 0.8550074792486326, 'colsample_bytree': 0.666306708438064}


In [70]:
xgb_best_model = xgb.XGBClassifier(n_estimators= 400,
                                   max_depth=5,
                                   learning_rate= 0.0152269895019286,
                                   subsample= 0.511642356633233,
                                   colsample_bytree= 0.9997371461265785)
#そのパラメータでモデルを新たに定義

In [71]:
xgb_best_model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9997371461265785, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0152269895019286,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [72]:
predictions = xgb_best_model.predict(x_test)
#x_testの結果を予測

In [73]:
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Transported': predictions
})
submission.head()
#submissionとして予測したデータを表示

,PassengerId,Transported
0,0013_01,1
1,0018_01,0
2,0019_01,1
3,0021_01,1
4,0023_01,1


In [ ]:
submission['Transported'] = submission['Transported'].astype(bool)
#Transportedをintからbooleanに変換

In [76]:
submission.to_csv('submission_xgb.csv', index=False)